In [1]:
import random
from datetime import datetime, timedelta


def generateSeeds(num=10):
    seeds = []
    for _ in range(num):
        timestamp = datetime.now() - timedelta(days=random.randint(1, 7))
        calls = 1
        tokens = random.randint(1000, 10000)
        spent = round(random.uniform(1.37, 5.62), 2)
        seeds.append((timestamp, calls, tokens, spent))

    return seeds

In [7]:
import sqlite3

def dbInteraction(data=None, filename="myDatabase.db", table="myTable", drop=False, clear=False, columns=None):
    # Connect to the SQLite database
    connection = sqlite3.connect(filename, detect_types=sqlite3.PARSE_DECLTYPES)
    cursor = connection.cursor()

    try:
        # Drop or clear the table if specified
        if drop:
            cursor.execute(f"DROP TABLE IF EXISTS {table}")
        elif clear:
            cursor.execute(f"DELETE FROM {table}")

        # If data is provided, create the table if it doesn't already exist and insert the data
        if data:
            # Construct the CREATE TABLE query dynamically based on column names
            if columns:
                colDefs = ", ".join([f"{col} {colType}" for col, colType in columns.items()])
                tableQuery = f"CREATE TABLE IF NOT EXISTS {table} ({colDefs})"
            else:
                print("ERROR: No columns provided for table creation. Please provide the columns parameter.")
                return # Exit the function if no columns are provided
            
            if len(data) == 1:
                cursor.execute(
                    f"""INSERT INTO {table} (timestamp, calls, tokens, spent) VALUES (?, ?, ?, ?)""",
                    data[0]
                )
            else:
                cursor.executemany(
                    f"""INSERT INTO {table} (timestamp, calls, tokens, spent) VALUES (?, ?, ?, ?)""",
                    data
                )

        # Commit the changes and close the connection
        connection.commit()
    
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        connection.rollback()
    
    finally:
        cursor.close()
        connection.close()
        print("Database successfully updated!")

In [8]:
dbSeeds = generateSeeds(20)

columns = {
    "timestamp": "DATETIME",
    "calls": "INTEGER",
    "tokens": "INTEGER",
    "spent": "REAL"
}

dbInteraction(data=dbSeeds, filename="sample.db", table="sample", columns=columns)

Database successfully updated!


C:\Users\jhawk\AppData\Local\Temp\ipykernel_6836\312451782.py:31: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.executemany(


In [11]:
import pandas as pd

usage = pd.read_sql_query("SELECT * FROM sample", sqlite3.connect("sample.db"))
usage.head(10)


,timestamp,calls,tokens,spent
0,2024-03-04 15:01:12.897365,1,2581,4.11
1,2024-03-04 15:01:12.897365,1,7180,4.87
2,2024-03-05 15:01:12.897365,1,1884,4.12
3,2024-03-07 15:01:12.897365,1,6839,1.54
4,2024-03-07 15:01:12.897365,1,4976,4.52
5,2024-03-05 15:01:12.897365,1,9088,1.61
6,2024-03-03 15:01:12.897365,1,2020,4.26
7,2024-03-07 15:01:12.897365,1,3846,1.75
8,2024-03-02 15:01:12.897365,1,3332,3.00
9,2024-03-02 15:01:12.897365,1,8985,2.48


In [13]:
# def calcUsage():
#     connection = sqlite3.connect("sample.db")
#     cursor = connection.cursor()

#     cursor.execute("SELECT SUM(calls) FROM sample")
#     totCalls = cursor.fetchone()[0]

#     cursor.execute("SELECT SUM(tokens) FROM sample")
#     totTokens = cursor.fetchone()[0]

#     cursor.execute("SELECT SUM(spent) FROM sample")
#     totSpent = cursor.fetchone()[0]

#     cursor.close()
#     connection.close()

#     return totCalls, totTokens, totSpent

usage["calls"].sum(), usage["tokens"].sum(), usage["spent"].sum()

(30, 183881, 98.11)

In [ ]:
totCalls, totTokens, totSpent = calcUsage()

print(f"Total calls: {totCalls}")
print(f"Total tokens: {totTokens}")
print(f"Total spent: {totSpent}")